# Automatic Differentiation 
Automatic differentiation (AD) aka algorithmic or computational differentiation, is a set of techniques to calculate **exact** derivatives of functions. It is neither symbolic nor finite difference like approximations. 

There are two main methods: "Forward Mode" and "Reverse Mode". Both have strengths and weeknesses

### Forward Mode Automatic Differentiation:

Consider two smooth functions f and g, near point a. The following expansion is the same for g. 
$$f(x) = f(a) + f'(a)(x-a) + O(x-a)^2$$
Let $\epsilon = x-a$, such that:
$$f(x) = f(a+\epsilon) = f(a) + \epsilon f'(a) + O(\epsilon^2)$$

Sum of the two functions:

$$\begin{aligned}(f+g)(x) &=f(x)+g(x)=f(a)+\epsilon f^{\prime}(a)+g(a)+\epsilon g^{\prime}(a) \\ &=[f(a)+g(a)]+\epsilon\left[f^{\prime}(a)+g^{\prime}(a)\right] \end{aligned}$$

And the product gives:
\begin{aligned}(f \cdot g)(x) &=f(x) \cdot g(x)=\left[f(a)+\epsilon f^{\prime}(a)\right] \cdot\left[g(a)+\epsilon g^{\prime}(a)\right] \\ &=[f(a) \cdot g(a)]+\epsilon\left[f(a) g^{\prime}(a)+g(a) f^{\prime}(a)\right] \end{aligned}

Thus:
\begin{aligned}(f+g)^{\prime}(a) &=f^{\prime}(a)+g^{\prime}(a) \\(f \cdot g)^{\prime}(a) &=f^{\prime}(a) g(a)+f(a) g^{\prime}(a) \end{aligned}

We also have chain rule:
\begin{aligned}(f \circ g)(x) &=f(g(x))=f\left(g(a)+\epsilon g^{\prime}(a)\right) \\ &=f(g(a))+\epsilon g^{\prime}(a) f^{\prime}(g(a)) \end{aligned}
$$(f \circ g)^{\prime}(a)=f^{\prime}(g(a)) g^{\prime}(a)$$

We can see which information about each function we need in order to calculate derivatives of their combinations. For each function f we need its value f(a) and its derivative f'(a). This is the only information we require in order to calculate the first derivative of any combination of functions.

### Lets do this on a computer:

In [1]:
#dual will represent a function where we can store f(a) and f'(a).
struct Dual{T}
    val::T   # value
    der::T  # derivative
end

In [2]:
#Overide base functions. Adding AD functionality right into the world of math functions. 

import Base: +, *, -, ^, exp

+(f::Dual, g::Dual) = Dual(f.val + g.val, f.der + g.der)
+(f::Dual, α::Number) = Dual(f.val + α, f.der)
+(α::Number, f::Dual) = f + α

+ (generic function with 169 methods)

But where did these come from? These are these quations, which come from above$$(f+g)(x) =[f(a)+g(a)]+\epsilon\left[f^{\prime}(a)+g^{\prime}(a)\right]$$
If g is a constant:
$$(f+g)(x) =[f(a)+g(a)]+\epsilon\left[f^{\prime}(a)+0\right]$$



In [3]:
# now the rest of the stuff...
-(f::Dual, g::Dual) = Dual(f.val - g.val, f.der - g.der)

*(f::Dual, g::Dual) = Dual(f.val*g.val, f.der*g.val + f.val*g.der)
*(α::Number, f::Dual) = Dual(f.val * α, f.der * α)
*(f::Dual, α::Number) = α * f


^(f::Dual, n::Integer) = Base.power_by_squaring(f, n)  # use repeated squaring for integer powers

exp(f::Dual) = Dual(exp(f.val), exp(f.val) * f.der)


exp (generic function with 14 methods)

In [4]:
f = Dual(3,4)
g = Dual(5,6)
f+g

Dual{Int64}(8, 10)

Nice, $(f+g)(a) = 8$ and $(f+g)'(a) = 10$!

In [5]:
f*g

Dual{Int64}(15, 38)

In [6]:
#we can even do more complicated things!
f*(g+g)^3

Dual{Int64}(3000, 14800)

### Differentiating arbitrary functions.

In [7]:
h(x) = x^2 + 2
a = 3;

Where h is a function of x, which we can think of as an identity function passed in. The identity function is $\iota$ and $\mathbf{1}$ is a ones function. Always gives 1. This seems like weird uneceessary stuff, but, its to formally identify us passing a constant in. We only know the derivative of operations of other derivatives. But defining it this way, we know the derivative and function of a constant.
$$ h = \iota^2 + 2*\mathbf{1}$$

In [8]:
#^ has a derivative of 1, value of a.
xx = Dual(a,1)

Dual{Int64}(3, 1)

$\text { since } \iota^{\prime}(a)=1 \text { for any } a$

In [9]:
h(xx)

Dual{Int64}(11, 6)

It works!

h(3) = 3^2 + 2 =  11

h'(3) = 2*3 =  6

In [10]:
#wrapping this in a function:
derivative(f,x) = f(Dual(x,one(x))).der

derivative (generic function with 1 method)

In [11]:
derivative(x -> 3x^5 + 2, 2)

240

#### We can also compute partial derivatives with this. 
$$f(x,y)$$
$$\frac{\partial f}{\partial x}|_{x=a}$$

In [12]:
ff(x,y) = x^2 + y
b = 4.0
a = 3.0
ff_1(x) = ff(x,b)
derivative(ff_1,a)

6.0

## Generalization of ForwardDiff to higher dimensions:
$$f(a+\epsilon)=f(a)+\nabla f(a) \cdot \epsilon+\mathcal{O}\left(\epsilon^{2}\right)$$
\begin{array}{c}(f+g)(a+\epsilon)=[f(a)+g(a)]+[\nabla f(a)+\nabla g(a)] \cdot \epsilon \\ (f \cdot g)(a+\epsilon)=[f(a)+\nabla f(a) \cdot \epsilon][g(a)+\nabla g(a) \cdot \epsilon] \\ =f(a) g(a)+[f(a) \nabla g(a)+g(a) \nabla f(a)] \cdot \epsilon\end{array}

In [13]:
using StaticArrays
struct MultiDual{N,T}
    val::T
    derivs::SVector{N,T}
end

import Base: +, *

function +(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val + g.val, f.derivs + g.derivs)
end

function *(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val * g.val, f.val .* g.derivs + g.val .* f.derivs)
end


* (generic function with 385 methods)

In [14]:
gg(x, y) = x*x*y + x + y

(a, b) = (1.0, 2.0)

xx = MultiDual(a, SVector(1.0, 0.0))
yy = MultiDual(b, SVector(0.0, 1.0))

gg(xx, yy)


MultiDual{2,Float64}(5.0, [5.0, 2.0])

# Julia has this stuff already!

In [132]:
using ForwardDiff
#even more complicated derivative based terms.

ForwardDiff.gradient( array -> ( (x, y) = array; x^2 * y + x*y ), [1, 2])

2-element Array{Int64,1}:
 6
 2

In [133]:
ForwardDiff.derivative( array -> ( (x) = array; x^2 + x ), 1)

3

In [134]:
ForwardDiff.jacobian( array -> ( (x, y) = array; [x^2 * y + x*y,3.0] ), [1, 2])

2×2 Array{Float64,2}:
 6.0  2.0
 0.0  0.0

In [135]:
ForwardDiff.hessian( array -> ( (x, y) = array; x^2 * y + x*y ), [1, 2])

2×2 Array{Int64,2}:
 4  3
 3  0

# A note on Julia that I didn't understand before...

In [136]:
#Two ways to define functions!
function test1(x,y)
    return x+y
end

test2(x,y) = x+y

#They do the same thing:
println(test1(3,1))
println(test2(3,1))


#We can also convert the output...
function test1_float(x,y)::Float64
    return x+y
end

test2_float(x,y)::Float64 = x+y

println(test1_float(3,1))
println(test2_float(3,1))

#Anonymous Functions
println("Anonmym functions")
println((x -> x+1)(3))

#two arguement anonyms function
println("two arg anonym:")
println(((x,y) -> x+y)(3,1))

#zero arguemtn anonyms
(()->println("Printed from a zero arg anonym function"))()

#multi line anonym function
println("Multilines with ;")
(array -> ((a1, a2) = array; println(a1+a2)))([3,1])

#Or we can define it by this way:
test3 = function (x)
    x+1
end

println(test3(3))

#mapping functions onto an array... 
#computes the funnction on each of the elements. 
println("Rounding:")
println(map(round, [1.2,3.5,1.7]))
println("adding 1")
println(map(x-> x+1, [3.0, 4.0, 5.0]))

#Ok this is an anymoum function. which has two lines, decomposing a 2 element array then calculating 
#a func based on the two elements!
(xx -> ( (x, y) = xx; x^2 * y + x*y ))([3,4])

4
4
4.0
4.0
Anonmym functions
4
two arg anonym:
4
Printed from a zero arg anonym function
Multilines with ;
4
4
Rounding:
[1.0, 4.0, 2.0]
adding 1
[4.0, 5.0, 6.0]


48

# Reverse Mode

An alternative method to calculate derivatives is to fix not the variable with which to differentiate, but what it is that we differentiate.

To calculate the **adjoint** for each $i$, $$\bar{v}_{i}:=\frac{d f}{\partial v_{i}}$$

Thus, if $f = v_1 + v_2$ and $v_1 = v_3 + v_4$ and $v_2 = v_3 + v_5$


then:
$$\frac{\partial f}{\partial v_{3}}=\frac{\partial f}{\partial v_{1}} \frac{\partial v_{1}}{\partial v_{3}}+\frac{\partial f}{\partial v_{2}} \frac{\partial v_{2}}{\partial v_{3}}$$


$$\implies \overline{v_{3}}=\alpha_{13} \overline{v_{1}}+\alpha_{2,3} \overline{v_{2}}$$

where $\alpha_{i,j}$ is the coefficient specifying the relationship between the different terms. 

The adjoint information propagates down the graph, in reverse order, hense the name "reverse-mode"

For this reason, reverse mode is much harder to implement. However, it has the advantage taht all the derivatives $\frac{\partial f}{\partial x_i}$ are calculated in a single pass of the tree. Thus this is the method of choice for calculating the gradient of a function $\mathbb{R}^{n} \rightarrow \mathbb{R}$, which is a very common case, in the context of mathematical optimization and machine learning.

# Example of reverse mode

In general its' really hard to implement this in a general way, but easy to do by hand. Consider the function:
$$f(x,y,z) = xy - 2sin(xz)$$


In [144]:
ff(x,y,z) = x*y - 2*sin(x*z)

x,y,z = 1,2,3

v₁ = x
v₂ = y
v₃ = z
v₄ = v₁ * v₂ #x*y
v₅ = v₁ * v₃ #x*z
v₆ = sin(v₅) #sin(x*z)

v₇ = v₄ - 2v₆  # f

1.7177599838802655

In [145]:
ff(x, y, z)

1.7177599838802655

Ok, we have it decomposed, now we can calculate the adjoints. 

In [146]:
v̄₇ = 1   # seed
v̄₆ = -2 # ∂f/∂v₆ = ∂v₇/∂v₆
v̄₅ = v̄₆ * cos(v₅)  # ∂v₇/∂v₆ * ∂v₆/∂v₅
v̄₄ = 1   # ∂f/∂v₄ = ∂v₇/∂v₄
v̄₃ = v̄₅ * v₁  # ∂f/∂v₃ = ∂f/∂v₅ . ∂v₅/∂v₃. # This gives ∂f/∂z
v̄₂ = v̄₄ * v₁
v̄₁ = v̄₅*v₃ + v̄₄*v₂   # two *different* paths

7.939954979602673

In a single pass, we calculated the gradient, 

In [148]:
(v̄₁, v̄₂, v̄₃)

(7.939954979602673, 1, 1.9799849932008908)

In [150]:
#It's correct!
ForwardDiff.gradient(x->ff(x...), [x,y,z])

3-element Array{Float64,1}:
 7.939954979602673 
 1.0               
 1.9799849932008908